Goal: Demonstrate your ability to write tests and evaluate responses from large language
models.
Requirements:
● Create functions that perform specific tasks utilizing Google Gemini
● Write unit tests to ensure the functions perform as expected
● Use the Google Evaluation API to evaluate and compare functions using various
different prompt.

Challenge three: Instructions
1. Create a Jupyter Notebook using Vertex AI Colab Enterprise.
2. Create a Python function that uses Gemini to classify user questions into one of the following
categories: Employment, General Information, Emergency Services, or Tax Related
3. Create a second function that generates social media posts for government announcements like
weather emergencies, holidays, school closings, etc.
4. Write unit tests for each function using pytest.
5. Use the Google Evaluation API to evaluate and compare Gemini responses from different
prompts.
6. Upload the Notebook to GitHub for grading.


In [1]:
!pip install google-genai pydantic

In [2]:
!pip install pytest

In [3]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=hEpc0uz1X2Lw2XqLE0xelq9b1FSOJr&prompt=consent&token_

In [4]:
!pip install -q google-cloud-aiplatform[evaluation]

In [5]:
import os

# Correct way to set environment variables in Colab/Jupyter:
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'true'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'qwiklabs-gcp-03-b295c10c44aa'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

Function 1:

In [6]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
import json
import os

# --- Pydantic Schema (Same as before) ---
class QuestionClassification(BaseModel):
    """A structure to hold the question category."""
    category: str = Field(
        description="The classified category of the user's question. Must be one of: Employment, General Information, Emergency Services, or Tax Related."
    )

# --- Updated Classification Function ---
def classify_question_adc(user_question: str) -> str:
    """
    Classifies a user question using the Gemini API authenticated via ADC (Vertex AI).

    It automatically uses the ADC credentials configured via 'gcloud auth application-default login'.
    Requires GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION to be set in the environment.

    Args:
        user_question: The text of the question to be classified.

    Returns:
        The classified category as a string, or an error message.
    """
    try:
        # Initialize the client for Vertex AI.
        # It automatically finds ADC credentials and uses the project/location
        # from the GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION environment variables.
        client = genai.Client(
            vertexai=True
        )

        # 1. Define the classification prompt
        categories = "Employment, General Information, Emergency Services, or Tax Related"
        prompt = f"""
        Classify the following user question into one of the following categories: {categories}.
        Return only the category name in the required JSON format.

        User Question: "{user_question}"
        """

        # 2. Configure the model for structured JSON output
        config = types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=QuestionClassification,
        )

        # 3. Call the Gemini API
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[prompt],
            config=config,
        )

        # 4. Extract and return the category from the structured JSON
        response_data = json.loads(response.text)
        return response_data.get("category", "Classification Failed")

    except Exception as e:
        # This will now print more specific errors related to credentials or project setup
        return f"An error occurred: {e}"

# --- Example Usage ---
if __name__ == "__main__":
    if "GOOGLE_CLOUD_PROJECT" not in os.environ or "GOOGLE_CLOUD_LOCATION" not in os.environ:
        print("--- ERROR: Environment Variables Missing ---")
        print("Please set GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION environment variables.")
        print("1. Run 'gcloud auth application-default login' to set up ADC.")
        print("2. Set the environment variables in your terminal before running the script.")
    else:
        questions = [
            "My direct deposit didn't arrive this week.", # Employment
            "I need to report a fire at my apartment building.", # Emergency Services
            "What is the population of the city of Denver?", # General Information
            "Is the new tax form 1099-K mandatory for freelancers?", # Tax Related
        ]

        print("--- Question Classification Results (Authenticated via ADC) ---")
        for q in questions:
            category = classify_question_adc(q)
            print(f"Question: **{q}**\nCategory: **{category}**\n")

--- Question Classification Results (Authenticated via ADC) ---
Question: **My direct deposit didn't arrive this week.**
Category: **Employment**

Question: **I need to report a fire at my apartment building.**
Category: **Emergency Services**

Question: **What is the population of the city of Denver?**
Category: **General Information**

Question: **Is the new tax form 1099-K mandatory for freelancers?**
Category: **Tax Related**



Code to create package for gov classifier:

In [7]:
%%writefile gov_classifier.py
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
import json
import os

# --- Pydantic Schema ---
class QuestionClassification(BaseModel):
    """A structure to hold the question category."""
    category: str = Field(
        description="The classified category of the user's question. Must be one of: Employment, General Information, Emergency Services, or Tax Related."
    )

# --- Classification Function ---
def classify_question_adc(user_question: str) -> str:
    """
    Classifies a user question using the Gemini API authenticated via ADC (Vertex AI).

    It automatically uses the ADC credentials configured via 'gcloud auth application-default login'.
    Requires GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION to be set in the environment.

    Args:
        user_question: The text of the question to be classified.

    Returns:
        The classified category as a string, or an error message.
    """
    try:
        # Initialize the client for Vertex AI.
        client = genai.Client(
            vertexai=True
        )

        # 1. Define the classification prompt
        categories = "Employment, General Information, Emergency Services, or Tax Related"
        prompt = f"""
        Classify the following user question into one of the following categories: {categories}.
        Return only the category name in the required JSON format.

        User Question: "{user_question}"
        """

        # 2. Configure the model for structured JSON output
        config = types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=QuestionClassification,
        )

        # 3. Call the Gemini API
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[prompt],
            config=config,
        )

        # 4. Extract and return the category from the structured JSON
        response_data = json.loads(response.text)
        return response_data.get("category", "Classification Failed")

    except Exception as e:
        return f"An error occurred: {e}"

Overwriting gov_classifier.py


Code to create unit test for gov classifier:

In [8]:
%%writefile test_gov_classifier.py
import pytest
import json
from unittest.mock import patch, Mock
import sys

# Import the function and pydantic model from the classifier file
from gov_classifier import classify_question_adc, QuestionClassification

# --- Mock Classes ---

class MockResponse:
    """A mock object to simulate the API response object."""
    def __init__(self, text):
        self.text = text

# --- Test Cases ---

@patch('gov_classifier.genai')
def test_successful_classification(mock_genai):
    """Tests the happy path: successful client init and structured JSON response."""

    expected_category = "Employment"
    mock_json_output = json.dumps({"category": expected_category})

    # 1. Setup Mock API Client and Response
    mock_genai.Client.return_value.models.generate_content.return_value = MockResponse(mock_json_output)

    # 2. Call the function under test
    result = classify_question_adc("When is payday?")

    # 3. Assertions
    assert result == expected_category

    # Verify the client was initialized correctly for Vertex AI
    mock_genai.Client.assert_called_once_with(vertexai=True)

    # Verify the API call was made and structured output was configured
    call_kwargs = mock_genai.Client.return_value.models.generate_content.call_args[1]

    # Check config for structured output
    config = call_kwargs['config']
    assert config.response_mime_type == "application/json"
    assert config.response_schema == QuestionClassification


@patch('gov_classifier.genai')
def test_client_initialization_error(mock_genai):
    """Tests failure when genai.Client raises an exception (e.g., due to bad ADC setup)."""

    # 1. Setup Mock Client to raise an exception
    mock_genai.Client.side_effect = RuntimeError("ADC failed to find credentials.")

    # 2. Call the function
    result = classify_question_adc("How do I pay taxes?")

    # 3. Assertion
    assert "An error occurred" in result
    assert "ADC failed to find credentials." in result


@patch('gov_classifier.genai')
def test_api_call_error(mock_genai):
    """Tests failure when client.models.generate_content raises an exception (e.g., rate limit)."""

    # 1. Setup Mock Client and API Call to raise an exception
    mock_genai.Client.return_value.models.generate_content.side_effect = TimeoutError("API timed out.")

    # 2. Call the function
    result = classify_question_adc("I saw smoke, what should I do?")

    # 3. Assertion
    assert "An error occurred" in result
    assert "API timed out." in result


@patch('gov_classifier.genai')
def test_malformed_json_response(mock_genai):
    """Tests failure when the model returns valid JSON but with the wrong schema/key."""

    # 1. Setup Mock API Client with a response that is missing the 'category' key
    malformed_json = json.dumps({"incorrect_key": "Tax"})
    mock_genai.Client.return_value.models.generate_content.return_value = MockResponse(malformed_json)

    # 2. Call the function
    result = classify_question_adc("What is the capital city?")

    # 3. Assertion
    # The code relies on response_data.get("category", "Classification Failed")
    assert result == "Classification Failed"

Overwriting test_gov_classifier.py


Actually test gov classifer

In [9]:
!pytest test_gov_classifier.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content
plugins: typeguard-4.4.4, langsmith-0.4.33, anyio-4.11.0
collected 4 items                                                              

test_gov_classifier.py ....                                              [100%]

============================== 4 passed in 5.22s ===============================


**********************************************************



**********************************************************

Function 2:

In [10]:
import os
from google import genai
from google.genai import types

def generate_gov_post_vertex_ai(
    announcement_type: str,
    details: str,
    key_info: list[str],
    audience: str = "General Public",
    platform: str = "X (Twitter)",
    model_name: str = "gemini-2.5-flash"
) -> str:
    """
    Generates a social media post for government announcements using Gemini
    via Vertex AI, relying on Application Default Credentials (ADC) for auth.

    Args:
        announcement_type: The category of the announcement (e.g., "Weather Emergency").
        details: A brief summary or context.
        key_info: A list of critical points to include.
        audience: The target audience.
        platform: The social media platform.
        model_name: The Gemini model to use.

    Returns:
        The generated social media post as a string.
    """

    # --- 1. Get Authentication Parameters from Environment ---
    vertex_project = os.environ.get('GOOGLE_CLOUD_PROJECT')
    vertex_location = os.environ.get('GOOGLE_CLOUD_LOCATION')

    if not vertex_project or not vertex_location:
        return (
            "ERROR: Authentication setup failed. Please ensure both "
            "'GOOGLE_CLOUD_PROJECT' and 'GOOGLE_CLOUD_LOCATION' environment variables are set "
            "and that you have run 'gcloud auth application-default login'."
        )

    # --- 2. Initialize the Client with Explicit Vertex AI Configuration ---
    try:
        # Explicitly setting vertexai=True, project, and location ensures
        # the client uses the correct API endpoint and ADC for authentication.
        client = genai.Client(
            vertexai=True,
            project=vertex_project,
            location=vertex_location
        )
    except Exception as e:
        return f"Error initializing Gemini client for Vertex AI. Error: {e}"

    # --- 3. Define the System Instruction for consistent government voice ---
    system_instruction = (
        "You are an official government communications specialist. "
        "Your tone must be **clear, concise, professional, and authoritative**. "
        "The post must be tailored to the specified platform and target audience. "
        "Use appropriate emojis and relevant hashtags to maximize reach. "
        "The primary goal is to clearly and quickly convey critical information and actions."
    )

    # --- 4. Construct the main prompt ---
    key_info_str = "\n".join([f"- {item}" for item in key_info])

    prompt = f"""
    Generate a social media post for a **{announcement_type}** announcement.

    - **Platform:** {platform}
    - **Target Audience:** {audience}
    - **Context/Summary:** {details}
    - **Critical Information to Include (MUST include all):** {key_info_str}

    The final output should be ONLY the generated post content, ready to publish.
    """

    # --- 5. Configure the model ---
    config = types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7,
    )

    # --- 6. Call the Gemini API ---
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=[prompt],
            config=config,
        )
        return response.text
    except Exception as e:
        return f"An error occurred during API call: {e}"

# --- Example Usage ---

if __name__ == '__main__':
    # REMINDER: You MUST run these commands in your terminal *before* running this script:
    # 1. gcloud auth application-default login
    # 2. export GOOGLE_CLOUD_PROJECT='your-project-id'
    # 3. export GOOGLE_CLOUD_LOCATION='us-central1'

    print("## 🚧 Public Works - Road Closure Post (X/Twitter) 🚧")
    road_post = generate_gov_post_vertex_ai(
        announcement_type="Planned Road Closure",
        details="Maintenance work on the main bridge requires a full closure.",
        key_info=[
            "I-75 Northbound closure at Exit 195 (Main St).",
            "Closure is from 10 PM Friday to 5 AM Monday.",
            "Detour signs are posted redirecting traffic to US-41.",
            "Please plan alternate routes and expect delays."
        ],
        platform="X (Twitter)",
        audience="Commuters and Local Residents"
    )
    print(road_post)

## 🚧 Public Works - Road Closure Post (X/Twitter) 🚧
⚠️ TRAFFIC ALERT ⚠️ PLANNED ROAD CLOSURE

I-75 Northbound at Exit 195 (Main St) will be **fully closed** for essential bridge maintenance.

🗓️ **When:** Friday 10 PM - Monday 5 AM
➡️ **Detour:** Follow posted signs to US-41.

Commuters & local residents: Please plan alternate routes and expect delays.

#RoadClosure #I75 #TrafficAlert #Commute


In [11]:
import pytest
from unittest.mock import patch, Mock
import sys

# Add the source directory to the path for importing the module
# This ensures it can find the function even if run outside the module folder.
# In a standard setup, you would install the package, but for Colab/simple file testing, this works.
sys.path.append('.')
from social_post_generator import generate_gov_post_vertex_ai


# Define constants for the successful mock case
MOCK_PROJECT = "test-project-id"
MOCK_LOCATION = "us-central1"
MOCK_GENERATED_TEXT = "📢 ALERT: Test Post Generated Successfully!"

# --- Fixtures and Mocks ---

@pytest.fixture
def mock_success_response():
    """Mock object representing a successful API response."""
    # Create a mock response object with a 'text' attribute
    mock_response = Mock()
    mock_response.text = MOCK_GENERATED_TEXT
    return mock_response

@pytest.fixture
def mock_environment_variables():
    """A dictionary representing the required environment variables."""
    return {
        'GOOGLE_CLOUD_PROJECT': MOCK_PROJECT,
        'GOOGLE_CLOUD_LOCATION': MOCK_LOCATION,
    }

# --- Test Cases ---

@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_successful_post_generation(mock_os, mock_genai, mock_success_response, mock_environment_variables):
    """Tests the happy path: all environment variables are set and the API call succeeds."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock API Client and Response
    # This targets client.models.generate_content
    mock_genai.Client.return_value.models.generate_content.return_value = mock_success_response

    # 3. Call the function under test
    result = generate_gov_post_vertex_ai(
        announcement_type="Test Announcement",
        details="Test details for the post.",
        key_info=["Point 1", "Point 2"],
    )

    # 4. Assertions
    assert result == MOCK_GENERATED_TEXT

    # Verify the client was initialized correctly
    mock_genai.Client.assert_called_once_with(
        vertexai=True,
        project=MOCK_PROJECT,
        location=MOCK_LOCATION
    )

    # Verify the API call was made
    mock_genai.Client.return_value.models.generate_content.assert_called_once()

    # Verify the system instruction was included in the config
    call_args, call_kwargs = mock_genai.Client.return_value.models.generate_content.call_args
    config = call_kwargs['config']
    assert "You are an official government communications specialist" in config.system_instruction


@patch('social_post_generator.os')
def test_missing_project_env_var(mock_os):
    """Tests failure when GOOGLE_CLOUD_PROJECT is missing."""
    # Mock os.environ.get to return location but not project
    def mock_env_get(key):
        return None if key == 'GOOGLE_CLOUD_PROJECT' else 'us-central1'

    mock_os.environ.get.side_effect = mock_env_get

    # Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # Assertion
    assert "ERROR: Authentication setup failed" in result
    assert "'GOOGLE_CLOUD_PROJECT'" in result


@patch('social_post_generator.os')
def test_missing_location_env_var(mock_os):
    """Tests failure when GOOGLE_CLOUD_LOCATION is missing."""
    # Mock os.environ.get to return project but not location
    def mock_env_get(key):
        return None if key == 'GOOGLE_CLOUD_LOCATION' else 'test-project-id'

    mock_os.environ.get.side_effect = mock_env_get

    # Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # Assertion
    assert "ERROR: Authentication setup failed" in result
    assert "'GOOGLE_CLOUD_LOCATION'" in result


@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_client_initialization_error(mock_os, mock_genai, mock_environment_variables):
    """Tests failure when genai.Client raises an exception during initialization (Step 2)."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock Client to raise an exception
    mock_genai.Client.side_effect = Exception("Auth key invalid.")

    # 3. Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # 4. Assertion
    assert "Error initializing Gemini client for Vertex AI" in result
    assert "Auth key invalid." in result


@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_api_call_error(mock_os, mock_genai, mock_environment_variables):
    """Tests failure when client.models.generate_content raises an exception (Step 6)."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock API Call to raise an exception
    mock_genai.Client.return_value.models.generate_content.side_effect = Exception("Rate limit exceeded.")

    # 3. Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # 4. Assertion
    assert "An error occurred during API call" in result
    assert "Rate limit exceeded." in result

Code to create social post generator package

In [12]:
%%writefile social_post_generator.py
import os
from google import genai
from google.genai import types

def generate_gov_post_vertex_ai(
    announcement_type: str,
    details: str,
    key_info: list[str],
    audience: str = "General Public",
    platform: str = "X (Twitter)",
    model_name: str = "gemini-2.5-flash"
) -> str:
    """
    Generates a social media post for government announcements using Gemini
    via Vertex AI, relying on Application Default Credentials (ADC) for auth.

    Args:
        announcement_type: The category of the announcement (e.g., "Weather Emergency").
        details: A brief summary or context.
        key_info: A list of critical points to include.
        audience: The target audience.
        platform: The social media platform.
        model_name: The Gemini model to use.

    Returns:
        The generated social media post as a string.
    """

    # --- 1. Get Authentication Parameters from Environment ---
    vertex_project = os.environ.get('GOOGLE_CLOUD_PROJECT')
    vertex_location = os.environ.get('GOOGLE_CLOUD_LOCATION')

    if not vertex_project or not vertex_location:
        return (
            "ERROR: Authentication setup failed. Please ensure both "
            "'GOOGLE_CLOUD_PROJECT' and 'GOOGLE_CLOUD_LOCATION' environment variables are set "
            "and that you have run 'gcloud auth application-default login'."
        )

    # --- 2. Initialize the Client with Explicit Vertex AI Configuration ---
    try:
        # Explicitly setting vertexai=True, project, and location ensures
        # the client uses the correct API endpoint and ADC for authentication.
        client = genai.Client(
            vertexai=True,
            project=vertex_project,
            location=vertex_location
        )
    except Exception as e:
        return f"Error initializing Gemini client for Vertex AI. Error: {e}"

    # --- 3. Define the System Instruction for consistent government voice ---
    system_instruction = (
        "You are an official government communications specialist. "
        "Your tone must be **clear, concise, professional, and authoritative**. "
        "The post must be tailored to the specified platform and target audience. "
        "Use appropriate emojis and relevant hashtags to maximize reach. "
        "The primary goal is to clearly and quickly convey critical information and actions."
    )

    # --- 4. Construct the main prompt ---
    key_info_str = "\n".join([f"- {item}" for item in key_info])

    prompt = f"""
    Generate a social media post for a **{announcement_type}** announcement.

    - **Platform:** {platform}
    - **Target Audience:** {audience}
    - **Context/Summary:** {details}
    - **Critical Information to Include (MUST include all):** {key_info_str}

    The final output should be ONLY the generated post content, ready to publish.
    """

    # --- 5. Configure the model ---
    config = types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7,
    )

    # --- 6. Call the Gemini API ---
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=[prompt],
            config=config,
        )
        return response.text
    except Exception as e:
        return f"An error occurred during API call: {e}"

Overwriting social_post_generator.py


Code to create unit test for social post generator;

In [13]:
%%writefile test_social_post_generator.py
import pytest
from unittest.mock import patch, Mock
import sys

# The module is in the same directory, so it should import directly.
from social_post_generator import generate_gov_post_vertex_ai


# Define constants for the successful mock case
MOCK_PROJECT = "test-project-id"
MOCK_LOCATION = "us-central1"
MOCK_GENERATED_TEXT = "📢 ALERT: Test Post Generated Successfully!"

# --- Fixtures and Mocks ---

@pytest.fixture
def mock_success_response():
    """Mock object representing a successful API response."""
    # Create a mock response object with a 'text' attribute
    mock_response = Mock()
    mock_response.text = MOCK_GENERATED_TEXT
    return mock_response

@pytest.fixture
def mock_environment_variables():
    """A dictionary representing the required environment variables."""
    return {
        'GOOGLE_CLOUD_PROJECT': MOCK_PROJECT,
        'GOOGLE_CLOUD_LOCATION': MOCK_LOCATION,
    }

# --- Test Cases ---

@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_successful_post_generation(mock_os, mock_genai, mock_success_response, mock_environment_variables):
    """Tests the happy path: all environment variables are set and the API call succeeds."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock API Client and Response
    # This targets client.models.generate_content
    mock_genai.Client.return_value.models.generate_content.return_value = mock_success_response

    # 3. Call the function under test
    result = generate_gov_post_vertex_ai(
        announcement_type="Test Announcement",
        details="Test details for the post.",
        key_info=["Point 1", "Point 2"],
    )

    # 4. Assertions
    assert result == MOCK_GENERATED_TEXT

    # Verify the client was initialized correctly
    mock_genai.Client.assert_called_once_with(
        vertexai=True,
        project=MOCK_PROJECT,
        location=MOCK_LOCATION
    )

    # Verify the API call was made
    mock_genai.Client.return_value.models.generate_content.assert_called_once()

    # Verify the system instruction was included in the config
    call_args, call_kwargs = mock_genai.Client.return_value.models.generate_content.call_args
    config = call_kwargs['config']
    assert "You are an official government communications specialist" in config.system_instruction


@patch('social_post_generator.os')
def test_missing_project_env_var(mock_os):
    """Tests failure when GOOGLE_CLOUD_PROJECT is missing."""
    # Mock os.environ.get to return location but not project
    def mock_env_get(key):
        return None if key == 'GOOGLE_CLOUD_PROJECT' else 'us-central1'

    mock_os.environ.get.side_effect = mock_env_get

    # Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # Assertion
    assert "ERROR: Authentication setup failed" in result
    assert "'GOOGLE_CLOUD_PROJECT'" in result


@patch('social_post_generator.os')
def test_missing_location_env_var(mock_os):
    """Tests failure when GOOGLE_CLOUD_LOCATION is missing."""
    # Mock os.environ.get to return project but not location
    def mock_env_get(key):
        return None if key == 'GOOGLE_CLOUD_LOCATION' else 'test-project-id'

    mock_os.environ.get.side_effect = mock_env_get

    # Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # Assertion
    assert "ERROR: Authentication setup failed" in result
    assert "'GOOGLE_CLOUD_LOCATION'" in result


@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_client_initialization_error(mock_os, mock_genai, mock_environment_variables):
    """Tests failure when genai.Client raises an exception during initialization (Step 2)."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock Client to raise an exception
    mock_genai.Client.side_effect = Exception("Auth key invalid.")

    # 3. Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # 4. Assertion
    assert "Error initializing Gemini client for Vertex AI" in result
    assert "Auth key invalid." in result


@patch('social_post_generator.genai')
@patch('social_post_generator.os')
def test_api_call_error(mock_os, mock_genai, mock_environment_variables):
    """Tests failure when client.models.generate_content raises an exception (Step 6)."""

    # 1. Setup Mock Environment Variables
    mock_os.environ.get.side_effect = lambda k: mock_environment_variables.get(k)

    # 2. Setup Mock API Call to raise an exception
    mock_genai.Client.return_value.models.generate_content.side_effect = Exception("Rate limit exceeded.")

    # 3. Call the function
    result = generate_gov_post_vertex_ai("A", "B", ["C"])

    # 4. Assertion
    assert "An error occurred during API call" in result
    assert "Rate limit exceeded." in result

Overwriting test_social_post_generator.py


In [14]:
!pytest test_social_post_generator.py


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content
plugins: typeguard-4.4.4, langsmith-0.4.33, anyio-4.11.0
collected 5 items                                                              

test_social_post_generator.py .....                                      [100%]

============================== 5 passed in 5.05s ===============================


Use the Google Evaluation API to evaluate and compare Gemini responses from different
prompts.

In [9]:
from vertexai.evaluation import EvalTask, PairwiseMetric


In [10]:
!pip install --upgrade google-cloud-aiplatform

  Using cached google_cloud_aiplatform-1.127.0-py2.py3-none-any.whl.metadata (46 kB)
Using cached google_cloud_aiplatform-1.127.0-py2.py3-none-any.whl (8.1 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.71.1
    Uninstalling google-cloud-aiplatform-1.71.1:
      Successfully uninstalled google-cloud-aiplatform-1.71.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.127.0 which is incompatible.


In [2]:
!pip uninstall vertexai google-cloud-aiplatform -y

Found existing installation: vertexai 1.71.1
Uninstalling vertexai-1.71.1:
  Successfully uninstalled vertexai-1.71.1
Found existing installation: google-cloud-aiplatform 1.71.1
Uninstalling google-cloud-aiplatform-1.71.1:
  Successfully uninstalled google-cloud-aiplatform-1.71.1


In [4]:
!pip install --upgrade vertexai==1.71.1 google-cloud-aiplatform


  Using cached vertexai-1.71.1-py3-none-any.whl.metadata (10 kB)
  Using cached google_cloud_aiplatform-1.127.0-py2.py3-none-any.whl.metadata (46 kB)
  Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached vertexai-1.71.1-py3-none-any.whl (7.3 kB)
Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl (6.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.14.1 requires google-cloud-aiplatform[agent-engines]<2.0.0,>=1.95.1, but you have google-cloud-aiplatform 1.71.1 which is incompatible.


In [1]:
import vertexai
import google.cloud.aiplatform as aiplatform

print("vertexai:", vertexai.__version__)
print("aiplatform:", aiplatform.__version__)



vertexai: 1.127.0
aiplatform: 1.127.0


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [14]:
from vertexai.evaluation import EvalTask, PairwiseMetric
import pandas as pd
from google.cloud import aiplatform
import os

PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION", "us-central1")
OUTPUT_BUCKET = "gs://qwiklabs-gcp-03-b295c10c44aa-eval-bucket"  # Must exist in your project

aiplatform.init(project=PROJECT_ID, location=LOCATION)

evaluation_inputs = [
    {
        "input": "The city council approved a 5% property tax increase effective Jan 1st.",
        "output_a": "Detailed announcement about the 5% property tax increase.",
        "output_b": "Concise social media post: 5% property tax hike approved."
    },
    {
        "input": "Due to a water main break, the intersection of Main and 5th street is closed indefinitely. Use detours.",
        "output_a": "Formal notice: Main & 5th closed due to water main break.",
        "output_b": "Social media alert: Main & 5th closed. Detours in effect."
    }
]

dataset_df = pd.DataFrame(evaluation_inputs)

# Define pairwise metric
pairwise_metric = PairwiseMetric(
    metric="pairwise",
    metric_prompt_template="Which output is more authoritative and concise? 0=worst, 1=best."
)

# EvalTask with GCS output path
eval_task = EvalTask(
    dataset=dataset_df,
    metrics=[pairwise_metric],
    output_uri_prefix=OUTPUT_BUCKET
)

# Run the evaluation
# For pairwise comparison, we don't need to generate outputs (we already have output_a and output_b)
# Just run the evaluation metrics on the existing data
results = eval_task.evaluate()

print("✅ Evaluation complete.")
print(f"View results in GCS: {OUTPUT_BUCKET}")

# ============================================
# CORRECT WAYS TO ACCESS RESULTS
# ============================================

# Method 1: Access summary_metrics (aggregated scores)
print("\n📊 Summary Metrics:")
print(results.summary_metrics)

# Method 2: Access metrics_table (detailed per-row results)
print("\n📋 Detailed Metrics Table:")
metrics_df = results.metrics_table
print(metrics_df)

# Method 3: Save to CSV for analysis
metrics_df.to_csv("evaluation_results.csv", index=False)
print("\n💾 Results saved to evaluation_results.csv")

# Method 4: Access specific columns
if 'pairwise/pairwise_choice' in metrics_df.columns:
    print("\n🎯 Pairwise Choices:")
    print(metrics_df[['input', 'pairwise/pairwise_choice']])

# Method 5: Calculate custom statistics
if 'pairwise/pairwise_choice' in metrics_df.columns:
    choice_counts = metrics_df['pairwise/pairwise_choice'].value_counts()
    print("\n📈 Choice Distribution:")
    print(choice_counts)

# Method 6: Access individual metric results (if needed)
print("\n🔍 Available attributes in results:")
print([attr for attr in dir(results) if not attr.startswith('_')])

INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 2/2 [00:09<00:00,  4.74s/it]
INFO:vertexai.evaluation._evaluation:All 2 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:9.492955125999288 seconds


✅ Evaluation complete.
View results in GCS: gs://qwiklabs-gcp-03-b295c10c44aa-eval-bucket

📊 Summary Metrics:
{'row_count': 2, 'pairwise/candidate_model_win_rate': np.float64(0.0), 'pairwise/baseline_model_win_rate': np.float64(0.0)}

📋 Detailed Metrics Table:
                                               input  \
0  The city council approved a 5% property tax in...   
1  Due to a water main break, the intersection of...   

                                            output_a  \
0  Detailed announcement about the 5% property ta...   
1  Formal notice: Main & 5th closed due to water ...   

                                            output_b  \
0  Concise social media post: 5% property tax hik...   
1  Social media alert: Main & 5th closed. Detours...   

                                pairwise/explanation pairwise/pairwise_choice  
0  Outputs A and B were not provided, so a compar...                      TIE  
1  Outputs (A) and (B) were not provided for comp...                    